# RNN with tensorflow2.0

In [2]:
import tensorflow as tf
import numpy as np

Limited tf.compat.v2.summary API due to missing TensorBoard installation
Limited tf.summary API due to missing TensorBoard installation


## Be sure to used Tensorflow 2.0

In [3]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

## Open and process dataset

In [4]:
# You can used your own dataset with english text

with open("rnn_dataset/victorhugo.txt", "r") as f:
    text = f.read()

print(len(text))
print(len(text))

print(text[:1000])


127286
127286
Parce que, jargonnant vêpres, jeûne et vigile,
Exploitant Dieu qui rêve au fond du firmament,
Vous avez, au milieu du divin évangile,
Ouvert boutique effrontément ;

Parce que vous feriez prendre à Jésus la verge,
Cyniques brocanteurs sortis on ne sait d'où ;
Parce que vous allez vendant la sainte vierge
Dix sous avec miracle, et sans miracle un sou ;

Parce que vous contez d'effroyables sornettes
Qui font des temples saints trembler les vieux piliers ;
Parce que votre style éblouit les lunettes
Des duègnes et des marguilliers ;

Parce que la soutane est sous vos redingotes,
Parce que vous sentez la crasse et non l'œillet,
Parce que vous bâclez un journal de bigotes
Pensé par Escobar, écrit par Patouillet ;

Parce qu'en balayant leurs portes, les concierges
Poussent dans le ruisseau ce pamphlet méprisé ;
Parce que vous mêlez à la cire des cierges
Votre affreux suif vert-de-grisé ;

Parce qu'à vous tout seuls vous faites une espèce
Parce qu'enfin, blanchis dehors et noirs 

In [5]:
import unidecode

text = unidecode.unidecode(text)
text = text.lower()

text = text.replace("2", "")
text = text.replace("1", "")
text = text.replace("8", "")
text = text.replace("5", "")
text = text.replace(">", "")
text = text.replace("<", "")
text = text.replace("!", "")
text = text.replace("?", "")
text = text.replace("-", "")
text = text.replace("$", "")

text = text.strip()

vocab = set(text)
print(len(vocab), vocab)

print(text[:100])


34 {'v', 'd', 'r', 'k', 'x', '"', 'y', 'g', 'b', ',', 'n', 'm', 'q', 'u', 'p', 's', '.', 'w', 'i', ':', 'o', 'l', 'e', 'j', '\n', 't', ' ', 'a', 'f', 'z', 'h', "'", ';', 'c'}
parce que, jargonnant vepres, jeune et vigile,
exploitant dieu qui reve au fond du firmament,
vous a


In [6]:
vocab_size = len(vocab)

vocab_to_int = {l:i for i,l in enumerate(vocab)}
int_to_vocab = {i:l for i,l in enumerate(vocab)}

print("vocab_to_int", vocab_to_int)
print()
print("int_to_vocab", int_to_vocab)

print("\nint for e", vocab_to_int["e"])
print("\nletter for e", vocab_to_int["e"])

vocab_to_int {'v': 0, 'd': 1, 'r': 2, 'k': 3, 'x': 4, '"': 5, 'y': 6, 'g': 7, 'b': 8, ',': 9, 'n': 10, 'm': 11, 'q': 12, 'u': 13, 'p': 14, 's': 15, '.': 16, 'w': 17, 'i': 18, ':': 19, 'o': 20, 'l': 21, 'e': 22, 'j': 23, '\n': 24, 't': 25, ' ': 26, 'a': 27, 'f': 28, 'z': 29, 'h': 30, "'": 31, ';': 32, 'c': 33}

int_to_vocab {0: 'v', 1: 'd', 2: 'r', 3: 'k', 4: 'x', 5: '"', 6: 'y', 7: 'g', 8: 'b', 9: ',', 10: 'n', 11: 'm', 12: 'q', 13: 'u', 14: 'p', 15: 's', 16: '.', 17: 'w', 18: 'i', 19: ':', 20: 'o', 21: 'l', 22: 'e', 23: 'j', 24: '\n', 25: 't', 26: ' ', 27: 'a', 28: 'f', 29: 'z', 30: 'h', 31: "'", 32: ';', 33: 'c'}

int for e 22

letter for e 22


In [7]:
encoded = [vocab_to_int[l] for l in text]
encoded_sentence = encoded[:100]

print(encoded_sentence)

[14, 27, 2, 33, 22, 26, 12, 13, 22, 9, 26, 23, 27, 2, 7, 20, 10, 10, 27, 10, 25, 26, 0, 22, 14, 2, 22, 15, 9, 26, 23, 22, 13, 10, 22, 26, 22, 25, 26, 0, 18, 7, 18, 21, 22, 9, 24, 22, 4, 14, 21, 20, 18, 25, 27, 10, 25, 26, 1, 18, 22, 13, 26, 12, 13, 18, 26, 2, 22, 0, 22, 26, 27, 13, 26, 28, 20, 10, 1, 26, 1, 13, 26, 28, 18, 2, 11, 27, 11, 22, 10, 25, 9, 24, 0, 20, 13, 15, 26, 27]


In [8]:
decoded_sentence = [int_to_vocab[i] for i in encoded_sentence]
print(decoded_sentence)

['p', 'a', 'r', 'c', 'e', ' ', 'q', 'u', 'e', ',', ' ', 'j', 'a', 'r', 'g', 'o', 'n', 'n', 'a', 'n', 't', ' ', 'v', 'e', 'p', 'r', 'e', 's', ',', ' ', 'j', 'e', 'u', 'n', 'e', ' ', 'e', 't', ' ', 'v', 'i', 'g', 'i', 'l', 'e', ',', '\n', 'e', 'x', 'p', 'l', 'o', 'i', 't', 'a', 'n', 't', ' ', 'd', 'i', 'e', 'u', ' ', 'q', 'u', 'i', ' ', 'r', 'e', 'v', 'e', ' ', 'a', 'u', ' ', 'f', 'o', 'n', 'd', ' ', 'd', 'u', ' ', 'f', 'i', 'r', 'm', 'a', 'm', 'e', 'n', 't', ',', '\n', 'v', 'o', 'u', 's', ' ', 'a']


In [9]:
decoded_sentence = "".join(decoded_sentence)
print(decoded_sentence)

parce que, jargonnant vepres, jeune et vigile,
exploitant dieu qui reve au fond du firmament,
vous a


In [10]:
inputs, targets = encoded, encoded[1:]

print("Inputs", inputs[:10])
print("Targets", targets[:10])

Inputs [14, 27, 2, 33, 22, 26, 12, 13, 22, 9]
Targets [27, 2, 33, 22, 26, 12, 13, 22, 9, 26]


## Method used to generate batch in sequence order

In [11]:
def gen_batch(inputs, targets, seq_len, batch_size, noise=0):
    # Size of each chunk
    chuck_size = (len(inputs) -1)  // batch_size
    # Numbef of sequence per chunk
    sequences_per_chunk = chuck_size // seq_len

    for s in range(0, sequences_per_chunk):
        batch_inputs = np.zeros((batch_size, seq_len))
        batch_targets = np.zeros((batch_size, seq_len))
        for b in range(0, batch_size):
            fr = (b*chuck_size)+(s*seq_len)
            to = fr+seq_len
            batch_inputs[b] = inputs[fr:to]
            batch_targets[b] = inputs[fr+1:to+1]
            
            if noise > 0:
                noise_indices = np.random.choice(seq_len, noise)
                batch_inputs[b][noise_indices] = np.random.randint(0, vocab_size)
            
        yield batch_inputs, batch_targets

for batch_inputs, batch_targets in gen_batch(inputs, targets, 5, 32, noise=0):
    print(batch_inputs[0], batch_targets[0])
    break
for batch_inputs, batch_targets in gen_batch(inputs, targets, 5, 32, noise=3):
    print(batch_inputs[0], batch_targets[0])
    break

[14. 27.  2. 33. 22.] [27.  2. 33. 22. 26.]
[24. 24.  2. 24. 22.] [27.  2. 33. 22. 26.]


## Create your own layer

In [12]:
class OneHot(tf.keras.layers.Layer):
    def __init__(self, depth, **kwargs):
        super(OneHot, self).__init__(**kwargs)
        self.depth = depth

    def call(self, x, mask=None):
        return tf.one_hot(tf.cast(x, tf.int32), self.depth)

Test if the layer works well

In [13]:
class RnnModel(tf.keras.Model):

    def __init__(self, vocab_size):
        super(RnnModel, self).__init__()
        # Convolutions
        self.one_hot = OneHot(len(vocab))

    def call(self, inputs):
        output = self.one_hot(inputs)
        return output

batch_inputs, batch_targets = next(gen_batch(inputs, targets, 50, 32))
model = RnnModel(len(vocab))
output = model.predict(batch_inputs)[0][0]

print("Input letter is:", batch_inputs[0][0])
print("One hot representation of the letter", output)

assert(output[int(batch_inputs[0][0])]==1)


Input letter is: 14.0
One hot representation of the letter [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Set up the model

In [14]:
vocab_size = len(vocab)

### Creat the layers

# Set the input of the model
tf_inputs = tf.keras.Input(shape=(None,), batch_size=64)
# Convert each value of the  input into a one encoding vector
one_hot = OneHot(len(vocab))(tf_inputs)
# Stack LSTM cells
rnn_layer1 = tf.keras.layers.LSTM(128, return_sequences=True, stateful=True)(one_hot)
rnn_layer2 = tf.keras.layers.LSTM(128, return_sequences=True, stateful=True)(rnn_layer1)
# Create the outputs of the model
hidden_layer = tf.keras.layers.Dense(128, activation="relu")(rnn_layer2)
outputs = tf.keras.layers.Dense(vocab_size, activation="softmax")(hidden_layer)

### Setup the model
model = tf.keras.Model(inputs=tf_inputs, outputs=outputs)

## Check if we can reset the RNN cells

In [15]:
# Star by resetting the cells of the RNN
model.reset_states()

# Get one batch
batch_inputs, batch_targets = next(gen_batch(inputs, targets, 50, 64))

# Make a first prediction
outputs = model.predict(batch_inputs)
first_prediction = outputs[0][0]

# Reset the states of the RNN states
model.reset_states()

# Make an other prediction to check the difference
outputs = model.predict(batch_inputs)
second_prediction = outputs[0][0]

# Check if both prediction are equal
assert(set(first_prediction)==set(second_prediction))

## Set the loss and objectives

In [16]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.001)

## Set some metrics to track the progress of the training

In [17]:
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## Set the train method and the predict method in graph mode

In [18]:
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(inputs)
        # Get the error/loss on these predictions
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

@tf.function
def predict(inputs):
    # Make a prediction on all the batch
    predictions = model(inputs)
    return predictions

# Train the model

In [ ]:
model.reset_states()

for epoch in range(50000):
    for batch_inputs, batch_targets in gen_batch(inputs, targets, 100, 64, noise=20):
        train_step(batch_inputs, batch_targets)
    template = '\r Epoch {}, Train Loss: {}, Train Accuracy: {}'
    print(template.format(epoch, train_loss.result(), train_accuracy.result()*100), end="")
    model.reset_states()

# Generate some text

In [35]:
import random

model.reset_states()

size_poetries = 300

poetries = np.zeros((64, size_poetries, 1))
sequences = np.zeros((64, 100))
for b in range(64):
    rd = np.random.randint(0, len(inputs) - 100)
    sequences[b] = inputs[rd:rd+100]

for i in range(size_poetries+1):
    if i > 0:
        poetries[:,i-1,:] = sequences
    softmax = predict(sequences)
    # Set the next sequences
    sequences = np.zeros((64, 1))
    for b in range(64):
        argsort = np.argsort(softmax[b][0])
        argsort = argsort[::-1]
        # Select one of the strongest 4 proposals
        sequences[b] = argsort[0]

for b in range(64):
    sentence = "".join([int_to_vocab[i[0]] for i in poetries[b]])
    print(sentence)
    print("\n=====================\n")
        

 qu'il vole a la fiss un fond du ciel profonde,
sur la presse, l'affecait l'echaux se decondee
te fon, le confone la chacant de sa fait,
l'heure est en vavant, ma cloire, l'eperance 
les amants, que sur l'homme en porte le menatte ;
vous avez quelque chose en vous qui vous defend d'honsement.
ces fe


ntre de l'amour les petits sur la parrie
il plass doux et nous calayds que tu navres,
par les vieux nous chontez pas un chanteur d'hommes
bui, la nuit, que jamais le ponde qui nous leurre,
nous n'aumez que jamais  que face de votre bouche,
et que j'avoidais la couph dans les beaux, me meme ;
je suis


 :
o mon ame en seuille d'une action manteau 

t 
out commencements n'est pous de tout hais,
fourmis. croisser la foit des fleurs  des lyneraples 
le coucher ces joups fans les flots dans les bants sont la perrhe ;
fes vieux bances cent et des fains doufoir trembles,
n'us comprends quit ces foeurs  


re sage, a tes piedres aux boulets
rentre de l'autel, voila quels sont vos refes
et quand l

In [25]:
import json
model.save("model_victor_hugo.h5")

with open("model_victor_hugo_vocab_to_int", "w") as f:
    f.write(json.dumps(vocab_to_int))
with open("model_victor_hugo_int_to_vocab", "w") as f:
    f.write(json.dumps(int_to_vocab))

In [34]:
import json

with open("model_rnn_vocab_to_int", "r") as f:
    vocab_to_int = json.loads(f.read())
with open("model_rnn_int_to_vocab", "r") as f:
    int_to_vocab = json.loads(f.read())
    int_to_vocab = {int(key):int_to_vocab[key] for key in int_to_vocab}

model.load_weights("model_rnn.h5")